# How to train the Baseline Models for the SENSORIUM track

### This notebook will show how to
- instantiate dataloader for the Sensorium track
- instantiate pytorch model
- instantiate a trainer function
- train two baselines for this competition track
- save the model weights (the model weights can already be found in './model_checkpoints/pretrained/')

### Imports

In [1]:
import torch
torch.cuda.set_device('cuda:3')

In [2]:
import datajoint as dj
dj.__version__

'0.13.7'

In [2]:
import torch
import numpy as np
import pandas as pd

#import matplotlib.pyplot as plt
#import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from nnfabrik.builder import get_data, get_model, get_trainer

In [4]:
!python --version

Python 3.10.15


### Instantiate DataLoader

In [3]:
# loading the SENSORIUM dataset
filenames = ['../data/1_25_13_12_3_2',]#['../data/static26872-17-20-GrayImageNet-94c6ff995dac583098847cfecd43e7b6.zip', ]

dataset_fn = 'sensorium.datasets.static_loaders'
dataset_config = {'paths': filenames,
                 'normalize': True,
                 'include_behavior': False,
                 'include_eye_position': False,
                 'batch_size': 128,
                 'scale':0.25,
                 }

dataloaders = get_data(dataset_fn, dataset_config)

# Instantiate State of the Art Model (SOTA)

In [4]:
model_fn = 'sensorium.models.ecker_core_full_gauss_readout'#'sensorium.models.stacked_core_full_gauss_readout'
model_config = {'pad_input': False,
  'stack': -1,
  'layers': 4,
  'input_kern': 9,
  'gamma_input': 6.3831,
  'gamma_readout': 0.0076,
  'hidden_kern': 7,
  'hidden_channels': 128,
  'num_rotations': 8, 
  'depth_separable': True,
  'grid_mean_predictor': {'type': 'cortex',
   'input_dimensions': 2,
   'hidden_layers': 1,
   'hidden_features': 30,
   'final_tanh': True},
  'init_sigma': 0.1,
  'init_mu_range': 0.3,
  'gauss_type': 'full',
  'shifter': False,
  # 'core': 'RotEquiv2dCore',
}

model = get_model(model_fn=model_fn,
                  model_config=model_config,
                  dataloaders=dataloaders,
                  seed=42,)

In [7]:
model

FiringRateEncoder(
  (core): Stacked2dCore(
    (_input_weights_regularizer): LaplaceL2norm(
      (laplace): Laplace()
    )
    (features): Sequential(
      (layer0): Sequential(
        (conv): Conv2d(1, 128, kernel_size=(9, 9), stride=(1, 1), bias=False)
        (norm): BatchNorm2d(128, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
        (nonlin): AdaptiveELU()
      )
      (layer1): Sequential(
        (ds_conv): DepthSeparableConv2d(
          (in_depth_conv): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1))
          (spatial_conv): Conv2d(128, 128, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=128)
          (out_depth_conv): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1))
        )
        (norm): BatchNorm2d(128, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
        (nonlin): AdaptiveELU()
      )
      (layer2): Sequential(
        (ds_conv): DepthSeparableConv2d(
          (in_depth_conv): Conv2d(128, 128, kernel_si

In [8]:
import neuralpredictors

## Configure Trainer

In [5]:
trainer_fn = "sensorium.training.standard_trainer"

trainer_config = {'max_iter': 200,
                 'verbose': False,
                 'lr_decay_steps': 4,
                 'avg_loss': False,
                 'lr_init': 0.009,
                 }

trainer = get_trainer(trainer_fn=trainer_fn, 
                     trainer_config=trainer_config)

# Run model training

In [6]:
validation_score, trainer_output, state_dict = trainer(model, dataloaders, seed=42)

Epoch 1:  29%|██▊       | 10/35 [00:02<00:06,  4.08it/s]


KeyboardInterrupt: 

### Save model checkpoints after training is complete

In [ ]:
torch.save(model.state_dict(), './model_checkpoints/sensorium_sota_model.pth')

## Load Model Checkpoints

In [5]:
model.load_state_dict(torch.load("./model_checkpoints/pretrained/sensorium_sota_model.pth"));

---

# Train a simple LN model

Our LN model has the same architecture as our CNN model (a convolutional core followed by a gaussian readout)
but with all non-linearities removed except the final ELU+1 nonlinearity.
Thus turning the CNN model effectively into a fully linear model followed by a single output non-linearity.


In [6]:
model_fn = 'sensorium.models.stacked_core_full_gauss_readout'
model_config = {'pad_input': False,
              'stack': -1,
              'layers': 3,
              'input_kern': 9,
              'gamma_input': 6.3831,
              'gamma_readout': 0.0076,
              'hidden_kern': 7,
              'hidden_channels': 64,
              'grid_mean_predictor': {'type': 'cortex',
              'input_dimensions': 2,
              'hidden_layers': 1,
              'hidden_features': 30,
              'final_tanh': True},
              'depth_separable': True,
              'init_sigma': 0.1,
              'init_mu_range': 0.3,
              'gauss_type': 'full',
              'linear': True
               }
model = get_model(model_fn=model_fn,
                  model_config=model_config,
                  dataloaders=dataloaders,
                  seed=42,)

In [ ]:
validation_score, trainer_output, state_dict = trainer(model, dataloaders, seed=42)

In [ ]:
torch.save(model.state_dict(), './model_checkpoints/sensorium_ln_model.pth')

In [7]:
model.load_state_dict(torch.load("./model_checkpoints/pretrained/sensorium_ln_model.pth"));

---